In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------  12.8/12.9 MB 89.2 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 42.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pyodbc

In [3]:
import os
import json
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Define the scope for Google Drive API
SCOPES = ['https://www.googleapis.com/auth/drive.file']

# Function to authenticate and get Google Drive service
def authenticate():
    creds = None
    token_file = 'token.json'

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open(token_file, 'w') as token:
            token.write(creds.to_json())

    return creds

# Function to upload a file to Google Drive
def upload_to_drive(file_path, file_name):
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype='text/csv')

    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f"File uploaded successfully. File ID: {file.get('id')}")

# Interactively specify the file to upload
csv_file_path = 'users.csv'
if not os.path.exists(csv_file_path):
    print(f"Error: File {csv_file_path} does not exist.")
else:
    upload_to_drive(csv_file_path, os.path.basename(csv_file_path))


Error: File users.csv does not exist.


In [ ]:
!pip show google-auth
!pip show google-auth-oauthlib
!pip show google-api-python-client

In [ ]:
!pip install google-auth google-auth-oauthlib google-api-python-client

In [ ]:
import sys
print(sys.executable)

In [ ]:
import pyodbc
import csv
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
import os

# Define connection parameters
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = 'guest'
sql_file = 'test.sql'
output_csv = 'query_results.csv'
credentials_file = 'credentials.json'

# Establish the connection
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password}'
    )
    print("Connection to SQL Server was successful.")
except Exception as e:
    print(f"Failed to connect to SQL Server: {e}")
    raise

# Read and execute the SQL query
try:
    with open(sql_file, 'r') as file:
        query = file.read()

    cursor = conn.cursor()
    cursor.execute(query)

    # Fetch results
    columns = [column[0] for column in cursor.description]  # Get column names
    rows = cursor.fetchall()

    # Write results to a CSV file
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)  # Write column headers
        writer.writerows(rows)  # Write data rows

    print(f"Results have been written to {output_csv}.")
except Exception as e:
    print(f"Error executing query: {e}")
finally:
    # Clean up and close the connection
    conn.close()
    print("Connection closed.")

# Upload to Google Drive
def upload_to_google_drive():
    # Authenticate and create the Drive API service
    SCOPES = ['https://www.googleapis.com/auth/drive.file']

    flow = InstalledAppFlow.from_client_secrets_file(credentials_file, SCOPES)
    creds = flow.run_local_server(port=0)
    service = build('drive', 'v3', credentials=creds)

    # Folder ID from the provided link
    folder_id = '1OqOktV99WyRrloXigw-UhxA7DuEQ8NFf'

    # Upload the file to the specified folder
    file_metadata = {
        'name': output_csv,
        'parents': [folder_id]  # Specify the parent folder
    }
    media = MediaFileUpload(output_csv, mimetype='text/csv')
    uploaded_file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print(f"File uploaded to Google Drive folder with ID: {uploaded_file.get('id')}")


# Call the upload function
try:
    upload_to_google_drive()
except Exception as e:
    print(f"Error uploading to Google Drive: {e}")


In [ ]:
!pip install google-cloud-bigquery


In [ ]:
import pyodbc
import csv
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
import os


# Define connection parameters
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = 'guest'
sql_file = 'test.sql'
credentials_file = 'credentials.json'
table_id = "ut-gee-wri-hydro-dev.test.test2"

# Establish the connection
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password}'
    )
    print("Connection to SQL Server was successful.")
except Exception as e:
    print(f"Failed to connect to SQL Server: {e}")
    raise

# Read and execute the SQL query
try:
    with open(sql_file, 'r') as file:
        query = file.read()

    cursor = conn.cursor()
    cursor.execute(query)

    # Fetch results
    columns = [column[0] for column in cursor.description]  # Get column names
    rows = cursor.fetchall()

    # Write results to a CSV file
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)  # Write column headers
        writer.writerows(rows)  # Write data rows

    print(f"Results have been written to {output_csv}")
except Exception as e:
    print(f"Error executing query: {e}")
finally:
    # Clean up and close the connection
    conn.close()
    print("Connection closed.")

### Query SQL Server to get Divrt Stations Metadata and Upload them to BigQuery

In [3]:
from google.cloud import bigquery  
from google.oauth2 import service_account  
import pyodbc
import csv
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
import os

# Define connection parameters
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = 'guest'
sql_file = 'test.sql'
credentials_file = 'credentials.json'
table_id = "ut-gee-wri-hydro-dev.test.Things"


# Path to your service account key
credentialsPath = r'ut-gee-wri-hydro-dev-e91fd0400fd0.json'  
credentials = service_account.Credentials.from_service_account_file(credentialsPath)  
client = bigquery.Client(credentials=credentials)  

# Define the table ID and file path


# Establish the connection to SQL Server
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password}'
    )
    print("Connection to SQL Server was successful.")
except Exception as e:
    print(f"Failed to connect to SQL Server: {e}")
    raise


# Read and execute the SQL query
try:
    with open(sql_file, 'r') as file:
        query = file.read()

    cursor = conn.cursor()
    cursor.execute(query)

    # Fetch results
    columns = [column[0] for column in cursor.description]  # Get column names
    rows = cursor.fetchall()  # Fetch all rows

    # Prepare data for BigQuery
    data = [dict(zip(columns, row)) for row in rows]
    
    # Define BigQuery job configuration
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite existing data
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON  # Use JSON format for rows
    )

    print("Query is done.")
    
    # Load data to BigQuery
    job = client.load_table_from_json(data, table_id, job_config=job_config)  # Load data directly from memory

    # Wait for the job to complete
    job.result()

    # Confirm the load
    table = client.get_table(table_id)  # Make an API request
    print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

except Exception as e:
    print(f"Error executing query or loading data to BigQuery: {e}")
finally:
    # Clean up and close the connection
    conn.close()
    print("Connection closed.")



Connection to SQL Server was successful.
Query is done.
Loaded 766 rows and 23 columns to ut-gee-wri-hydro-dev.test.Things
Connection closed.


### Query SQL Server to get Divrt Stations Data Streams Upload them to BigQuery

In [ ]:
from google.cloud import bigquery  
from google.oauth2 import service_account  
import pyodbc
import csv
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
import os

# Define connection parameters
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = 'guest'
sql_file = 'test.sql'
credentials_file = 'credentials.json'
table_id = "ut-gee-wri-hydro-dev.test.Datastreams"


# Path to your service account key
credentialsPath = r'ut-gee-wri-hydro-dev-e91fd0400fd0.json'  
credentials = service_account.Credentials.from_service_account_file(credentialsPath)  
client = bigquery.Client(credentials=credentials)  

# Define the table ID and file path


# Establish the connection to SQL Server
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password}'
    )
    print("Connection to SQL Server was successful.")
except Exception as e:
    print(f"Failed to connect to SQL Server: {e}")
    raise


# Read and execute the SQL query
try:
    with open(sql_file, 'r') as file:
        query = file.read()

    cursor = conn.cursor()
    cursor.execute(query)

    # Fetch results
    columns = [column[0] for column in cursor.description]  # Get column names
    rows = cursor.fetchall()  # Fetch all rows

    # Prepare data for BigQuery
    data = [dict(zip(columns, row)) for row in rows]
    
    # Define BigQuery job configuration
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite existing data
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON  # Use JSON format for rows
    )

    print("Query is done.")
    
    # Load data to BigQuery
    job = client.load_table_from_json(data, table_id, job_config=job_config)  # Load data directly from memory

    # Wait for the job to complete
    job.result()

    # Confirm the load
    table = client.get_table(table_id)  # Make an API request
    print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

except Exception as e:
    print(f"Error executing query or loading data to BigQuery: {e}")
finally:
    # Clean up and close the connection
    conn.close()
    print("Connection closed.")



### Query the Daily Records table and Upload it to BigQuery

In [ ]:
import pyodbc
from google.cloud import bigquery
from google.oauth2 import service_account

# Define connection parameters for the SQL Server
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = 'guest'
sql_file = 'DAILY_RECORDS.sql'

# BigQuery configuration
credentialsPath = r'ut-gee-wri-hydro-dev-e91fd0400fd0.json'
credentials = service_account.Credentials.from_service_account_file(credentialsPath)
client = bigquery.Client(credentials=credentials)
table_id = "ut-gee-wri-hydro-dev.test.DAILY_RECORDS"

# Establish the connection to SQL Server
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password}'
    )
    print("Connection to SQL Server was successful.")
except Exception as e:
    print(f"Failed to connect to SQL Server: {e}")
    raise

# Read and execute the SQL query
try:
    with open(sql_file, 'r') as file:
        query = file.read()

    cursor = conn.cursor()
    cursor.execute(query)

    # Fetch results
    columns = [column[0] for column in cursor.description]  # Get column names
    rows = cursor.fetchall()  # Fetch all rows

    # Prepare data for BigQuery
    data = [dict(zip(columns, row)) for row in rows]
    
    # Define BigQuery job configuration
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite existing data
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON  # Use JSON format for rows
    )

    print("Query is done.")
    
    # Load data to BigQuery
    job = client.load_table_from_json(data, table_id, job_config=job_config)  # Load data directly from memory

    # Wait for the job to complete
    job.result()

    # Confirm the load
    table = client.get_table(table_id)  # Make an API request
    print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

except Exception as e:
    print(f"Error executing query or loading data to BigQuery: {e}")
finally:
    # Clean up and close the connection
    conn.close()
    print("Connection closed.")






